# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 19 2022 10:17AM,237635,18,INNOVA-8040350,Innova Softgel LLC,Released
1,May 19 2022 10:11AM,237633,10,8037183,Eywa Pharma Inc.,Released
2,May 19 2022 10:11AM,237633,10,8037187,Eywa Pharma Inc.,Released
3,May 19 2022 10:11AM,237633,10,8037188,Eywa Pharma Inc.,Released
4,May 19 2022 10:11AM,237633,10,8037200,Eywa Pharma Inc.,Released
5,May 19 2022 10:11AM,237633,10,8037201,Eywa Pharma Inc.,Released
6,May 19 2022 10:11AM,237633,10,8037202,Eywa Pharma Inc.,Released
7,May 19 2022 10:11AM,237633,10,8037207,Eywa Pharma Inc.,Released
8,May 19 2022 10:11AM,237633,10,8037211,Eywa Pharma Inc.,Released
9,May 19 2022 10:11AM,237633,10,8037212,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
56,237631,Released,1
57,237632,Released,25
58,237633,Released,15
59,237634,Released,13
60,237635,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
237631,NaN,NaN,1.0
237632,NaN,NaN,25.0
237633,NaN,NaN,15.0
237634,NaN,NaN,13.0
237635,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237539,0.0,6.0,2.0
237540,0.0,9.0,4.0
237545,15.0,6.0,0.0
237550,0.0,0.0,1.0
237557,0.0,10.0,18.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237539,0,6,2
237540,0,9,4
237545,15,6,0
237550,0,0,1
237557,0,10,18


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237539,0,6,2
1,237540,0,9,4
2,237545,15,6,0
3,237550,0,0,1
4,237557,0,10,18


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237539,,6,2
1,237540,,9,4
2,237545,15,6,
3,237550,,,1
4,237557,,10,18


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 19 2022 10:17AM,237635,18,Innova Softgel LLC
1,May 19 2022 10:11AM,237633,10,Eywa Pharma Inc.
16,May 19 2022 10:11AM,237632,10,Eywa Pharma Inc.
41,May 19 2022 10:11AM,237634,20,Alumier Labs Inc.
54,May 19 2022 9:55AM,237631,19,ACG North America LLC
55,May 19 2022 9:49AM,237598,18,"Blinc International, Inc. - PVecom"
56,May 19 2022 9:49AM,237597,18,"Blinc International, Inc.-Blinc-ecom"
57,May 19 2022 9:40AM,237630,10,"E5 Pharma, LLC"
69,May 19 2022 9:37AM,237629,10,Eywa Pharma Inc.
70,May 19 2022 9:14AM,237624,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 19 2022 10:17AM,237635,18,Innova Softgel LLC,,,1
1,May 19 2022 10:11AM,237633,10,Eywa Pharma Inc.,,,15
2,May 19 2022 10:11AM,237632,10,Eywa Pharma Inc.,,,25
3,May 19 2022 10:11AM,237634,20,Alumier Labs Inc.,,,13
4,May 19 2022 9:55AM,237631,19,ACG North America LLC,,,1
5,May 19 2022 9:49AM,237598,18,"Blinc International, Inc. - PVecom",,,1
6,May 19 2022 9:49AM,237597,18,"Blinc International, Inc.-Blinc-ecom",,,1
7,May 19 2022 9:40AM,237630,10,"E5 Pharma, LLC",,,12
8,May 19 2022 9:37AM,237629,10,Eywa Pharma Inc.,,,1
9,May 19 2022 9:14AM,237624,10,ISDIN Corporation,,,54


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 19 2022 10:17AM,237635,18,Innova Softgel LLC,1,,
1,May 19 2022 10:11AM,237633,10,Eywa Pharma Inc.,15,,
2,May 19 2022 10:11AM,237632,10,Eywa Pharma Inc.,25,,
3,May 19 2022 10:11AM,237634,20,Alumier Labs Inc.,13,,
4,May 19 2022 9:55AM,237631,19,ACG North America LLC,1,,
5,May 19 2022 9:49AM,237598,18,"Blinc International, Inc. - PVecom",1,,
6,May 19 2022 9:49AM,237597,18,"Blinc International, Inc.-Blinc-ecom",1,,
7,May 19 2022 9:40AM,237630,10,"E5 Pharma, LLC",12,,
8,May 19 2022 9:37AM,237629,10,Eywa Pharma Inc.,1,,
9,May 19 2022 9:14AM,237624,10,ISDIN Corporation,54,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 19 2022 10:17AM,237635,18,Innova Softgel LLC,1,,
1,May 19 2022 10:11AM,237633,10,Eywa Pharma Inc.,15,,
2,May 19 2022 10:11AM,237632,10,Eywa Pharma Inc.,25,,
3,May 19 2022 10:11AM,237634,20,Alumier Labs Inc.,13,,
4,May 19 2022 9:55AM,237631,19,ACG North America LLC,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 19 2022 10:17AM,237635,18,Innova Softgel LLC,1.0,NaN,NaN
1,May 19 2022 10:11AM,237633,10,Eywa Pharma Inc.,15.0,NaN,NaN
2,May 19 2022 10:11AM,237632,10,Eywa Pharma Inc.,25.0,NaN,NaN
3,May 19 2022 10:11AM,237634,20,Alumier Labs Inc.,13.0,NaN,NaN
4,May 19 2022 9:55AM,237631,19,ACG North America LLC,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 19 2022 10:17AM,237635,18,Innova Softgel LLC,1.0,0.0,0.0
1,May 19 2022 10:11AM,237633,10,Eywa Pharma Inc.,15.0,0.0,0.0
2,May 19 2022 10:11AM,237632,10,Eywa Pharma Inc.,25.0,0.0,0.0
3,May 19 2022 10:11AM,237634,20,Alumier Labs Inc.,13.0,0.0,0.0
4,May 19 2022 9:55AM,237631,19,ACG North America LLC,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4039348,243.0,28.0,16.0
15,1425717,41.0,66.0,0.0
16,1425458,9.0,16.0,0.0
18,2138400,19.0,0.0,0.0
19,712858,16.0,29.0,0.0
20,1425497,90.0,34.0,0.0
21,237575,1.0,0.0,0.0
22,237571,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4039348,243.0,28.0,16.0
1,15,1425717,41.0,66.0,0.0
2,16,1425458,9.0,16.0,0.0
3,18,2138400,19.0,0.0,0.0
4,19,712858,16.0,29.0,0.0
5,20,1425497,90.0,34.0,0.0
6,21,237575,1.0,0.0,0.0
7,22,237571,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,243.0,28.0,16.0
1,15,41.0,66.0,0.0
2,16,9.0,16.0,0.0
3,18,19.0,0.0,0.0
4,19,16.0,29.0,0.0
5,20,90.0,34.0,0.0
6,21,1.0,0.0,0.0
7,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,243.0
1,15,Released,41.0
2,16,Released,9.0
3,18,Released,19.0
4,19,Released,16.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,21,22
Status,,,,,,,,
Completed,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,28.0,66.0,16.0,0.0,29.0,34.0,0.0,1.0
Released,243.0,41.0,9.0,19.0,16.0,90.0,1.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,21,22
0,Completed,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,28.0,66.0,16.0,0.0,29.0,34.0,0.0,1.0
2,Released,243.0,41.0,9.0,19.0,16.0,90.0,1.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,21,22
0,Completed,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,28.0,66.0,16.0,0.0,29.0,34.0,0.0,1.0
2,Released,243.0,41.0,9.0,19.0,16.0,90.0,1.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()